In [1]:
import jupyterlab_nodeeditor as jlne
import ipywidgets
from yggdrasil.runner import YggRunner

In [21]:
ne = jlne.NodeEditor()
ne

In [18]:
runner = YggRunner("yggdrasil_test_models/model_trifecta.yml")

In [16]:
runner.run()

INFO:21200:runner.startDrivers[804]:YggRunner(runner): Starting I/O drivers and models on system None in namespace yggdrasil with rank 0
X:\anaconda3\envs\yggJLNE\python.exe X:\College\Grad School\Research\gc-Xyzic\jupyterlab_nodeeditor\examples\wip_notebooks\yggdrasil_test_models\model_rng_test.py
X:\anaconda3\envs\yggJLNE\python.exe X:\College\Grad School\Research\gc-Xyzic\jupyterlab_nodeeditor\examples\wip_notebooks\yggdrasil_test_models\model_reader_test.py
X:\anaconda3\envs\yggJLNE\python.exe X:\College\Grad School\Research\gc-Xyzic\jupyterlab_nodeeditor\examples\wip_notebooks\yggdrasil_test_models\ygg_model_rotxx_test_model_rotxx.py
End of File
INFO:21200:runner.waitModels[871]:YggRunner(runner): model_rng finished running.
INFO:21200:runner.waitModels[877]:YggRunner(runner): model_rng finished exiting.
End of input from temp_x.
INFO:21200:runner.waitModels[871]:YggRunner(runner): model_rotxx finished running.
INFO:21200:runner.waitModels[877]:YggRunner(runner): model_rotxx finis

OrderedDict([('init', 1646785043.864637),
             ('load drivers', 1646785043.9051707),
             ('start drivers', 1646785044.0031934),
             ('run models', 1646785047.8809845),
             ('at exit', 1646785047.8889923)])

In [23]:
class CBJLNE:
    if len(ne.node_editor.nodes[0].display_element.children) <= 3:
        for i in range(3):
            ne.node_editor.nodes[i].display_element.children += ipywidgets.Widget("Initializing",)
    
    def __init__(self):
        self.label = ipywidgets.Label()
    
    def display_number(self, *args, **kwargs):
        ne.node_editor.nodes[0].display_element.children[3].value = f"Number: {args[0].args[1]}"
        
    def display_string(self, *args, **kwargs):
        ne.node_editor.nodes[1].display_element.children[3].value = f"String: {args[0].args[1]}"
        
    def display_rotated(self, *args, **kwargs):
        ne.node_editor.nodes[2].display_element.children[3].value = f"ROTXX: {args[0].args[1]}"
    
display = CBJLNE()

TypeError: __init__() takes 1 positional argument but 2 were given

In [ ]:
runner.connectiondrivers['model_reader:inputReader']['callbacks'] = [display.display_string]
runner.connectiondrivers['model_rng:outputRNG,model_reader:outputReader_to_model_rotxx:input_rotxx']['callbacks'] = [display.display_number]
runner.connectiondrivers['model_rotxx:output_rotxx']['callbacks'] = [display.display_rotated]